In [ ]:
import os
os.environ['config']='/home/ruyao/database.yaml'
from qsdata import kddb
import numpy as np
import pandas as pd
import datetime
import pickle
from qsdata.api import get_vwap, get_sw_industry
from datetime import date, timedelta
from qsdata.api import get_previous_trading_date,index_components,get_trading_dates,get_next_trading_date,get_price
import copy

In [ ]:
test_dt = ["20190417"]
for i in range(20):
    test_dt.append(get_next_trading_date(test_dt[-1]))
test_dt

In [ ]:
import random
from tqdm.notebook import tqdm
from multiprocessing import Process

In [ ]:
# 取沪深300和中证500的成分股
stockcodes = index_components('000905.SH', datetime.date(2019,1,1)) + index_components('000300.SH',datetime.date(2019,1,1))
features = ["open", "close", "high", "low", "turnover", "volume"]

In [ ]:
s_dt = "20180101"
e_dt = "20211001"
# s_dt = "20141101"
train_set = ["20180101", "20181231"]
valid_set = ["20190101", "20191231"]
test_set = ["20200101", "20211001"]

In [ ]:
'000049.sz' in stockcodes

## 获取日线数据
## Obain daily data

In [ ]:
# 获取日线数据
daily_data = get_price(None,
              start_date=s_dt,
              end_date=e_dt,
              fields=features,
              frequency='1d')

daily_data = daily_data.reset_index()

daily_data['dt'] = daily_data['dt'].apply(lambda x: x.date())
daily_data

In [ ]:
daily_data = daily_data.loc[(daily_data['turnover']>0) & (daily_data['volume']>0)]
daily_data

In [ ]:
# 对缺失日期进行补全，得到nan值
std_list = []

for s in stockcodes:
    for dt in get_trading_dates(s_dt,e_dt):
        std_dict = {'stock_code':s, 'dt':dt}
        std_list.append(std_dict)
std_df = pd.DataFrame(std_list)

daily_data = pd.merge(daily_data, std_df, on=['stock_code', 'dt'], how='outer')
daily_data = daily_data.sort_values(by=["stock_code", "dt"]).reset_index(drop=True)

## 构建标签
## Construct label

In [ ]:
daily_df = daily_data.copy()


In [ ]:
# 未来20天价格
for d in range(1, 21):
    daily_df['close_t' + str(d)] = daily_df.groupby('stock_code')['close'].shift(-d)
daily_df

In [ ]:
daily_df['close_t0'] = daily_df['close']
daily_df = daily_df.dropna().reset_index(drop=True)
daily_df

In [ ]:
daily_df.columns

In [ ]:
daily_df['5_day_rtn'] = daily_df['close_t5']/daily_df['close']

In [ ]:
daily_df['20_day_rtn'] = daily_df['close_t20']/daily_df['close']
daily_df

In [ ]:
# 计算夏普比率
for t in range(1, 21):
    base_close_label = "close" if t==1 else "close_t" + str(t-1)
    close_label = "close_t" + str(t)
    rtn_label = "rtn_t" + str(t)
    daily_df[rtn_label] = daily_df[close_label]/daily_df[base_close_label]
    
for SR_len in [5, 20]:
    att = ['rtn_t'+str(i) for i in range(1, SR_len+1)]
    daily_df['SR'+str(SR_len)] = daily_df[att].mean(axis=1)/daily_df[att].std(axis=1)

daily_df['close_rtn'] = daily_df['rtn_t1']
daily_df

In [ ]:
# SR排名的标签
# 分数从小到大排列
daily_df['close_rtn_rank'] = daily_df.groupby('dt')['close_rtn'].rank(method='min')/max(daily_df.groupby('dt')['close_rtn'].rank(method='min'))
daily_df['close_rtn_label'] = daily_df['close_rtn_rank'] .apply(lambda x: 1 if x>=2/3 else (0 if x<=1/3 else -1))


daily_df['SR5_rank'] = daily_df.groupby('dt')['SR5'].rank(method='min')/max(daily_df.groupby('dt')['SR5'].rank(method='min'))
daily_df['SR5_label'] = daily_df['SR5_rank'] .apply(lambda x: 1 if x>=2/3 else (0 if x<=1/3 else -1))

daily_df['SR20_rank'] = daily_df.groupby('dt')['SR20'].rank(method='min')/max(daily_df.groupby('dt')['SR20'].rank(method='min'))
daily_df['SR20_label'] = daily_df['SR20_rank'] .apply(lambda x: 1 if x>=2/3 else (0 if x<=1/3 else -1))
daily_df

In [ ]:
daily_df['5_day_rtn_rank'] = daily_df.groupby('dt')['5_day_rtn'].rank(method='min')/max(daily_df.groupby('dt')['5_day_rtn'].rank(method='min'))

In [ ]:
daily_df['20_day_rtn_rank'] = daily_df.groupby('dt')['20_day_rtn'].rank(method='min')/max(daily_df.groupby('dt')['20_day_rtn'].rank(method='min'))
daily_df

## 正则化
## Regularization

In [ ]:
def get_next_month(dt):
    if dt.month == 12:
        return date(dt.year + 1, 1, 1)
    else:
        return date(dt.year, dt.month + 1, 1)


def filter_extreme_MAD(series, n=5):  # MAD:中位数去极值
    median = np.percentile(series, 50)
    new_median = np.percentile((series - median).abs(), 50)
    max_range = median + n * new_median
    min_range = median - n * new_median
    return np.clip(series, min_range, max_range)


def filter_extreme_3sigma(series, n=3):  # 3 sigma
    mean = series.mean()
    std = series.std()
    max_range = mean + n * std
    min_range = mean - n * std
    return np.clip(series, min_range, max_range)


def filter_extreme_percentile(series, min=0.025, max=0.975):  # 百分位法
    series = series.sort_values()
    q = series.quantile([min, max])
    return np.clip(series, q.iloc[0], q.iloc[1])


def standardize_zscore(series):
    std = series.std()
    mean = series.mean()
    # 如果标准差为0的series，全部返回0
    if std == 0:
        return series - mean
    else:
        return (series - mean) / std


def standardize_normal(series):
    min_v = series.min()
    max_v = series.max()
    return (series - min_v) / (max_v - min_v)


def ds_filter_extreme_3sigma(series, n=3, min_range=None, max_range=None):  # 3 sigma
    if min_range is not None and max_range is not None:
        return np.clip(series, min_range, max_range), min_range, max_range
    else:
        mean = series.mean()
        std = series.std()
        max_range = mean + n * std
        min_range = mean - n * std
        return np.clip(series, min_range, max_range), min_range, max_range


def ds_filter_extreme_MAD(series, n=5, min_range=None, max_range=None):  # MAD:中位数去极值
    if min_range is not None and max_range is not None:
        return np.clip(series, min_range, max_range), min_range, max_range
    else:
        median = np.percentile(series, 50)
        new_median = np.percentile((series - median).abs(), 50)
        max_range = median + n * new_median
        min_range = median - n * new_median
        return np.clip(series, min_range, max_range), min_range, max_range


def ds_standardize_zscore(series, mean=None, std=None):
    if mean is not None and std is not None:
        if std != 0:
            return (series - mean) / std, mean, std
        else:
            return (series - mean), mean, std
    else:
        std = series.std()
        mean = series.mean()
        # 如果标准差为0的series，全部返回0
        if std == 0:
            return series - mean, mean, std
        else:
            return (series - mean) / std, mean, std
        
        
        
# 正则化

def normalization(daily_data, split_dt, labels=False, d_data=True, return_split=False):


    train_daily_df = daily_data[daily_data["dt"]<=split_dt]
    test_daily_df = daily_data[daily_data["dt"]>split_dt]
    
    if d_data:
        features = ["close","open","high","low","volume","turnover"]
    else:
        features = ["close","open","high","low","volume","amount"]
        
    if labels:
        features = labels
    
    for f in features:
        d = {}
        train_daily_df[f], min_range, max_range = ds_filter_extreme_3sigma(train_daily_df[f])
        train_daily_df[f], mean, std = ds_standardize_zscore(train_daily_df[f])
        d["min_range"], d['max_range'], d['mean'], d['std'] = min_range, max_range, mean, std
        test_daily_df[f], _, _ = ds_filter_extreme_MAD(test_daily_df[f], min_range = min_range, max_range = max_range)
        test_daily_df[f], _, _ = ds_standardize_zscore(test_daily_df[f], mean, std)

    print("成功去极值、标准化")
    
    if return_split:
        return train_daily_df, test_daily_df, d
    else:
        return pd.concat((train_daily_df, test_daily_df))
    


In [ ]:
# 去inf和nan值
def replace_inf(df, labels):
    for label in labels:
        df[label] = df[label].apply(lambda x: np.nan if np.isinf(x) else x)
    return df
daily_df = replace_inf(daily_df, ['close_rtn', 'SR5', 'SR20'])
daily_df = daily_df.dropna().reset_index(drop=True)
daily_df

In [ ]:
for f in ['SR5', 'SR20']:
    daily_df[f] = filter_extreme_percentile(daily_df[f], 0)

In [ ]:
daily_df['close_rtn'].describe()

In [ ]:
daily_df['SR5'].loc[daily_df['SR5']>113965]

In [ ]:
daily_df['SR5'].describe()

In [ ]:
daily_df['SR20'].describe()

In [ ]:
daily_df[daily_df['SR20']>129]

In [ ]:
# 标签标准化
labels = ['open', 'close', 'high', 'low', 'turnover', 'volume']

daily_df_normalized = normalization(daily_df, datetime.datetime.strptime(train_set[1], "%Y%m%d").date(), labels)
daily_df_normalized

# daily_feature_df_normalized = normalization(daily_feature_df, datetime.datetime.strptime(train_set[1], "%Y%m%d").date(), ['close_rtn', 'SR5', 'SR20'])
# daily_feature_df_normalized

### 数值分析 data analysis

In [ ]:
from matplotlib import pyplot as plt 

# test_open = daily_df_normalized['close_rtn'].value_counts()
# plt.scatter(test_open.index,test_open.values)
plt.hist(daily_df['SR5_label'])
daily_df_normalized['SR5_label'].describe()

In [ ]:
plt.hist(daily_df['SR20_label'])
daily_df_normalized['SR20_label'].describe()

In [ ]:
plt.hist(daily_df['close_rtn'])
daily_df_normalized['close_rtn'].describe()

In [ ]:
# train_d, test_d, d = normalization(daily_df, datetime.datetime.strptime(train_set[1], "%Y%m%d").date(), ['close_rtn', 'SR5', 'SR20'], return_split=True)
# d

## 构建日线数据
## Construct daily data

In [ ]:
daily_df_normalized = daily_df_normalized.dropna()
daily_df_normalized['features'] = daily_df_normalized[['open', 'close', 'high', 'low', 'turnover', 'volume']].apply(lambda x: list(x.values), axis=1)
daily_df_normalized

In [ ]:
# 对缺失日期进行补全，得到nan值
std_list = []

for s in stockcodes:
    for dt in get_trading_dates(s_dt,e_dt):
        std_dict = {'stock_code':s, 'dt':dt}
        std_list.append(std_dict)
std_df = pd.DataFrame(std_list)

daily_feature_df = pd.merge(daily_df_normalized, std_df, on=['stock_code', 'dt'], how='outer')
daily_feature_df = daily_feature_df.sort_values(by=["stock_code", "dt"]).reset_index(drop=True)
daily_feature_df

In [ ]:
# 构建20维
for i in range(1, 20):
    daily_feature_df['features-'+str(i)] = daily_feature_df.groupby('stock_code')['features'].shift(i)
    
daily_feature_df = daily_feature_df.dropna().reset_index(drop=True)
daily_feature_df

In [ ]:
daily_feature_df['daily20_features'] = daily_feature_df[['features']+['features-'+str(i) for i in range(1, 20)]].apply(lambda x: np.array(list(x.values)), axis=1)
daily_feature_df

In [ ]:
daily_feature_df['close_rtn'].describe()

In [ ]:
daily_feature_df.columns

In [ ]:
# 预测模型数据存储，最终需要的
daily_feature_df[['stock_code', 'dt', 'daily20_features', 'close_rtn', 'SR5', 'SR20', 'close_rtn_rank', 'close_rtn_label', 'SR5_rank',
       'SR5_label', 'SR20_rank', 'SR20_label', '5_day_rtn_rank', '20_day_rtn_rank']].to_pickle('/home/ruyao/self_supervised_model/data/daily_feature-2.pkl')

In [ ]:
daily_feature_df['stock_code'].value_counts().describe()

## Encoding模型数据建立-相关系数法

## Construct data

In [ ]:
daily_feature_df.columns

In [ ]:
pair_daily_df = daily_feature_df[['stock_code', 'dt','close_t0', 'close_t1', 'close_t2', 'close_t3', 'close_t4', 'close_t5', 'close_t6',
       'close_t7', 'close_t8', 'close_t9', 'close_t10', 'close_t11',
       'close_t12', 'close_t13', 'close_t14', 'close_t15', 'close_t16',
       'close_t17', 'close_t18', 'close_t19', 'close_t20', 'daily20_features']].copy()
# pair_daily_df['pair_stock_code'] = pair_daily_df['stock_code'].apply(lambda x: random.choice(stockcodes))
pair_daily_df['pair_stock_code'] = pair_daily_df['stock_code'].apply(lambda x: random.choice([k for k in stockcodes if k not in [x]]))

pair_daily_df

In [ ]:
pair_daily_df['close_array20'] = pair_daily_df[['close_t0', 'close_t1', 'close_t2', 'close_t3', 'close_t4', 'close_t5', 'close_t6',
       'close_t7', 'close_t8', 'close_t9', 'close_t10', 'close_t11',
       'close_t12', 'close_t13', 'close_t14', 'close_t15', 'close_t16',
       'close_t17', 'close_t18', 'close_t19', 'close_t20']].apply(lambda x: list(x.values), axis=1)
pair_daily_df['close_array5'] = pair_daily_df[['close_t0', 'close_t1', 'close_t2', 'close_t3', 'close_t4', 'close_t5']].apply(lambda x: list(x.values), axis=1)
pair_daily_df['close_array1'] = pair_daily_df[['close_t0', 'close_t1']].apply(lambda x: list(x.values), axis=1)

In [ ]:
pair_daily_df = pd.merge(pair_daily_df, pair_daily_df, left_on=["pair_stock_code", "dt"], right_on=["stock_code", "dt"])
pair_daily_df = pair_daily_df.dropna().reset_index(drop=True)
pair_daily_df

In [ ]:
pair_daily_df[['stock_code_x', 'stock_code_y', 'dt','daily20_features_x', 'daily20_features_y', 'close_array20_y']]

In [ ]:
import math
# 函数：计算相关系数
def calc_corr(a, b):
    a_avg = sum(a)/len(a)
    b_avg = sum(b)/len(b)
 
    # 计算分子，协方差————按照协方差公式，本来要除以n的，由于在相关系数中上下同时约去了n，于是可以不除以n
    cov_ab = sum([(x - a_avg)*(y - b_avg) for x,y in zip(a, b)])
 
    # 计算分母，方差乘积————方差本来也要除以n，在相关系数中上下同时约去了n，于是可以不除以n
    sum1 = sum([(x - a_avg)**2 for x in a])
    sum2 = sum([(x - b_avg)**2 for x in b])
    sq = math.sqrt(sum1*sum2)

    if sum1==0 and sum2==0:
        return 1.0
    elif sum1==0 or sum2==0:
        return 0.0
        
    corr_factor = cov_ab/sq    
 
    return corr_factor

In [ ]:
pair_daily_df['corr20'] = pair_daily_df.apply(lambda x: calc_corr(x['close_array20_x'], x['close_array20_y']), axis=1)
pair_daily_df['corr5'] = pair_daily_df.apply(lambda x: calc_corr(x['close_array5_x'], x['close_array5_y']), axis=1)

In [ ]:
pair_daily_df['corr1'] = pair_daily_df.apply(lambda x: calc_corr(x['close_array1_x'], x['close_array1_y']), axis=1)

In [ ]:
pair_daily_df[['corr20', 'corr5', 'corr1']]

In [ ]:
# 三分类标签，不相关0 正相关1 负相关2
pair_daily_df['corr1_label'] = pair_daily_df['corr1'] .apply(lambda x: 1 if x>=2/3 else (2 if x<=-1/3 else 0))
pair_daily_df['corr5_label'] = pair_daily_df['corr5'] .apply(lambda x: 1 if x>=0.5 else (2 if x<=-0.5 else 0))
pair_daily_df['corr20_label'] = pair_daily_df['corr20'] .apply(lambda x: 1 if x>=0.5 else (2 if x<=-0.5 else 0))

In [ ]:
q = pair_daily_df['corr5'].quantile([1/4, 3/4])
q

In [ ]:
pair_daily_df.columns

In [ ]:
pair_daily_df['corr1_label']

In [ ]:
# 存储encoding数据，最终使用的数据
pair_daily_df[['stock_code_x', 'stock_code_y', 'dt', 'daily20_features_x', 'daily20_features_y', 'corr20',
       'corr5', 'corr1', 'corr1_label', 'corr5_label', 'corr20_label']].to_pickle('/home/ruyao/self_supervised_model/data/encoding_feature.pkl')

In [ ]:
# 对label进行正则化

pair_daily_df_normalized = normalization(pair_daily_df, datetime.datetime.strptime(train_set[1], "%Y%m%d").date(), ['corr20', 'corr5', 'corr1'])
pair_daily_df_normalized

In [ ]:
# 存储encoding数据
# pair_daily_df_normalized[['stock_code_x', 'stock_code_y', 'dt', 'daily20_features_x', 'daily20_features_y', 'corr20',
#        'corr5', 'corr1']].to_pickle('/mnt/disk1/min_data/ruyao/v4/data/encoding_feature_normalized.pkl')


## Encoding模型数据建立 (v3)

In [ ]:
pair_daily_df = daily_feature_df[['stock_code', 'dt', 'rtn_t1', 'rtn_t2',
       'rtn_t3', 'rtn_t4', 'rtn_t5', 'rtn_t6', 'rtn_t7', 'rtn_t8', 'rtn_t9',
       'rtn_t10', 'rtn_t11', 'rtn_t12', 'rtn_t13', 'rtn_t14', 'rtn_t15',
       'rtn_t16', 'rtn_t17', 'rtn_t18', 'rtn_t19', 'rtn_t20','daily20_features']].copy()
# pair_daily_df['pair_stock_code'] = pair_daily_df['stock_code'].apply(lambda x: random.choice(stockcodes))
pair_daily_df['pair_stock_code'] = pair_daily_df['stock_code'].apply(lambda x: random.choice([k for k in stockcodes if k not in [x]]))

pair_daily_df

In [ ]:
pair_daily_df = pd.merge(pair_daily_df, pair_daily_df, left_on=["pair_stock_code", "dt"], right_on=["stock_code", "dt"])

pair_daily_df = pair_daily_df.dropna().reset_index(drop=True)
pair_daily_df

In [ ]:
def count_diff(label, n):
    pair_daily_df[label] = 0
    for i in range(1,n):    
        x = 'rtn_t'+str(i) + '_x'
        y = 'rtn_t'+str(i) + '_y'
        pair_daily_df[label] += pair_daily_df.apply(lambda df: 1 if ( (df[x]>1 and df[y]>1) or (df[x]<1 and df[y]<1) ) else 0, axis=1) # 1pair_df[x]//1) * pair_df[y]//1

count_diff('sum_rtn_diff20', 21)
count_diff('sum_rtn_diff', 6)

In [ ]:
pair_daily_df

In [ ]:
pair_daily_df['']

In [ ]:
pair_daily_df['co_rtn_label1'] = pair_daily_df.apply(lambda df: 1 if ( (df['rtn_t1_x']>1 and df['rtn_t1_y']>1) or (df['rtn_t1_x']<1 and df['rtn_t1_y']<1) ) else 0, axis=1)

pair_daily_df['co_rtn_label5'] = pair_daily_df['sum_rtn_diff'].apply(lambda x: 1 if x>=3 else 0)
pair_daily_df['co_rtn_label20'] = pair_daily_df['sum_rtn_diff20'].apply(lambda x: 1 if x>=13 else 0)
pair_daily_df

In [ ]:
pair_daily_df['co_rtn_label20'].describe()

In [ ]:
pair_daily_df.columns

In [ ]:
# 存储encoding数据
pair_daily_df[['stock_code_x', 'stock_code_y', 'dt', 'daily20_features_x', 'daily20_features_y', 'co_rtn_label1',
       'co_rtn_label5', 'co_rtn_label20']].to_pickle('/mnt/disk1/min_data/ruyao/v3/data/encoding_feature-2.pkl')